In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import sys
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
import time
from collections import Counter
tqdm.pandas(tqdm_notebook)
import lightgbm as lgb
import gc
import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression,Ridge,Lasso
import matplotlib.gridspec as gridspec
pd.set_option('display.float_format', '{:.5f}'.format)
SEED = 51
np.random.seed(SEED)
%matplotlib inline

In [2]:
train1 = pd.read_csv('train_binary_preds.csv')
train2 = pd.read_csv('train_outlierless.csv')
train3 = pd.read_csv('train_reg_preds.csv')
test1 = pd.read_csv('test_binary_preds.csv')
test2 = pd.read_csv('test_outlierless.csv')
test3 = pd.read_csv('test_reg_preds.csv')
#train1 = train1.sort_values(by = 'card_id',ascending=False)
#train2 = train2.sort_values(by = 'card_id',ascending=False)
#train3 = train3.sort_values(by = 'card_id',ascending=False)
train1 = train1.reset_index(drop=True)
train2 = train2.reset_index(drop=True)
train3 = train3.reset_index(drop=True)
print(train1.shape,train2.shape,train3.shape)
print(test1.shape,test2.shape,test3.shape)

(201917, 2) (201917, 2) (201917, 2)
(123623, 2) (123623, 2) (123623, 2)


In [3]:
train = pd.read_csv('../input/train.csv')
train = train.sort_values(by = 'card_id',ascending=False)
train = train.reset_index(drop=True)
y_ = train[['card_id','target']]

In [4]:
train_ = train1.merge(train2,how = 'left',on = 'card_id')
train_ = train_.merge(train3,how = 'left',on = 'card_id')
train_ = train_.merge(y_,how = 'left',on = 'card_id')

In [5]:
test = test1.merge(test2,how = 'left',on = 'card_id')
test = test.merge(test3,how = 'left',on = 'card_id')

In [11]:
y = train_['target']
train = train_[['card_id','binary_prob','without_outlier']]

In [12]:
feats = [c for c in train.columns if c not in ['card_id', 'first_active_month'] ]

In [13]:
stratified = False

if stratified:
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=SEED)
else:
    folds = KFold(n_splits= 5, shuffle=True, random_state=SEED)

# Create arrays and dataframes to store results
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()

In [14]:
# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [15]:
%%time
# k-fold
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
    train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
    valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]
    
    reg = LinearRegression(normalize=False)
    
    reg.fit(train_x,train_y)
    
    oof_preds[valid_idx] = reg.predict(valid_x)
    sub_preds += reg.predict(test[feats]) / folds.n_splits
    print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
    del reg, train_x, train_y, valid_x, valid_y
    gc.collect()
print("Average RMSE = {}".format(rmse(y,oof_preds)))

Fold  1 RMSE : 3.711541
Fold  2 RMSE : 3.571584
Fold  3 RMSE : 3.717328
Fold  4 RMSE : 3.599152
Fold  5 RMSE : 3.606067
Average RMSE = 3.6416450039106136
CPU times: user 1.02 s, sys: 1.45 s, total: 2.47 s
Wall time: 378 ms


In [16]:
test['target'] = sub_preds

In [17]:
submission_file_name = '../my_submissions/deneme5.csv'
test[['card_id', 'target']].to_csv(submission_file_name, index=False)